# Global Entities - Map Visualizer

#### Version 1.0

### TODO

- Plotly basic map plot
- Plotly with group level filter
- Plotly with entity filter for group
- Plotly lineplot/scatterplot/heatmap for selected entity
- Plotly Dash app

### Observations

## Setup

### Import Packages

In [52]:
from datetime import datetime, date
from pprint import pprint
from collections import OrderedDict
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Set ipython's max row display
# pd.set_option('display.max_row', 1000)
# Set iPython's max column width
pd.set_option('display.max_columns', 50)

sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = [20, 10]

### Functions

In [53]:
def masked_heatmap():
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    fig,ax = plt.subplots()
    sns.heatmap(corr,center = 0, fmt = ".3f", square = True, annot= True, mask=mask)

## Global Variables

#### Dataset Parameters

In [54]:
file_name = '../data/silver_tables/yearly_values_per_entity.csv'

year_range = [1960, 2022]
metric_agg = 'last_year'
metric_agg = 'growth_rate'
metric_agg = '?'

In [55]:
country_list = ['United Arab Emirates', 'Japan', 'Ghana'] #For country level analysis

In [56]:
key_columns = [
    'Annual CO2 emissions', 'Annual change in primary energy consumption (%)',
    'Per capita electricity (kWh)',
    'Fossil fuels per capita (kWh)', 'Fossil fuels (% equivalent primary energy)',
    'Renewables per capita (kWh - equivalent)', 'Renewables (% electricity)',
    'Solar Generation - TWh', 'prod of Electricity from solar (TWh)',
]

corr_columns = [
    'Annual CO2 emissions', 'Annual change in primary energy consumption (%)',
    'Per capita electricity (kWh)',
    'Fossil fuels per capita (kWh)',
    'Renewables per capita (kWh - equivalent)',
]

## Data Prep

In [57]:
df_original = pd.read_csv(file_name)

In [58]:
df_original = df_original.loc[
    (df_original['Year'] > year_range[0])
    & (df_original['Year'] < year_range[1])
]

In [59]:
df_original.head(2)

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels per capita (kWh),Oil Consumption - TWh,Economy,"Coal (TWh, direct energy)",Geo Biomass Other - TWh,Wind Generation - TWh,Fossil fuels (% equivalent primary energy),prod of Other renewables including bioenergy (TWh),Coal Production - TWh,Gas Production - TWh,"Gas (TWh, direct energy)",Per capita electricity (kWh),Renewables (% electricity),Annual CO2 emissions,Fossil fuels (TWh),Hydro Generation - TWh,Oil Production - TWh,Population density (people per sq. km of land area),Coal Consumption - TWh,Renewables per capita (kWh - equivalent),prod of Electricity from solar (TWh),Annual change in primary energy consumption (%),Solar Generation - TWh,Land area (sq. km),Gas Consumption - TWh,"Oil (TWh, direct energy)",prod of Electricity from hydro (TWh),Fossil fuels (% growth),prod of Electricity from wind (TWh),Entity_Category
34,Aruba,1961,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,645553.0,NaN,NaN,NaN,307.966667,NaN,NaN,NaN,NaN,NaN,180.0,NaN,NaN,NaN,NaN,NaN,Country
35,Aruba,1962,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,708942.0,NaN,NaN,NaN,312.411111,NaN,NaN,NaN,NaN,NaN,180.0,NaN,NaN,NaN,NaN,NaN,Country


In [60]:
df_original.describe()

,Year,Fossil fuels per capita (kWh),Oil Consumption - TWh,"Coal (TWh, direct energy)",Geo Biomass Other - TWh,Wind Generation - TWh,Fossil fuels (% equivalent primary energy),prod of Other renewables including bioenergy (TWh),Coal Production - TWh,Gas Production - TWh,"Gas (TWh, direct energy)",Per capita electricity (kWh),Renewables (% electricity),Annual CO2 emissions,Fossil fuels (TWh),Hydro Generation - TWh,Oil Production - TWh,Population density (people per sq. km of land area),Coal Consumption - TWh,Renewables per capita (kWh - equivalent),prod of Electricity from solar (TWh),Annual change in primary energy consumption (%),Solar Generation - TWh,Land area (sq. km),Gas Consumption - TWh,"Oil (TWh, direct energy)",prod of Electricity from hydro (TWh),Fossil fuels (% growth),prod of Electricity from wind (TWh)
count,13231.000000,4270.000000,4299.000000,57.000000,4230.000000,4230.000000,4270.000000,7050.000000,1397.000000,2514.000000,57.000000,5828.000000,5665.000000,1.265100e+04,4270.000000,4292.000000,2553.000000,11042.000000,4273.000000,4292.000000,7113.000000,9354.000000,4230.000000,1.197800e+04,4296.000000,57.000000,7178.000000,4255.000000,7113.000000
mean,1991.724662,32590.368226,1015.737954,28584.640556,5.525746,6.126806,86.307820,3.366912,1915.282764,941.158999,21492.190695,3924.760158,29.633481,2.164809e+08,2346.057603,62.145414,1707.803287,270.028485,756.214677,4.578573,1.344705,4.533664,2.250895,6.467467e+05,564.805312,39160.740543,38.012262,3.538322,3.658208
std,17.303559,34472.620328,4706.367669,10007.255369,34.808495,61.993965,15.585076,27.092075,6167.601614,3615.938174,9564.826179,5117.133819,32.033628,1.755023e+09,11082.105692,308.745114,5858.493747,1308.026452,3785.186997,13.474483,22.090646,31.757811,28.611929,1.812242e+06,2785.492795,8988.199100,240.564170,26.995583,47.898591
min,1961.000000,155.807388,0.097381,16060.809570,0.000000,0.000000,12.804697,0.000000,0.339596,0.000000,6303.798828,0.000000,0.000000,3.664000e+03,0.097381,0.000000,0.000000,0.098625,0.000000,0.000000,0.000000,-95.005081,0.000000,1.000000e+01,0.000000,17989.603516,0.000000,-49.590828,0.000000
25%,1977.000000,11678.774170,65.349743,20363.539062,0.000000,0.000000,80.717112,0.000000,37.385506,45.443999,14118.032227,572.427170,1.366416,6.012190e+05,117.750929,0.430750,102.900040,19.113774,3.395960,0.172228,0.000000,-0.781557,0.000000,1.827000e+04,8.854822,33680.054688,0.017904,-1.007167,0.000000
50%,1992.000000,25363.719727,142.951569,25963.681641,0.046381,0.000000,91.689793,0.000000,125.336014,144.380302,20265.484375,2469.344605,16.124605,5.064147e+06,283.652481,4.616636,344.998383,63.247178,32.584167,0.806088,0.000000,2.361172,0.000000,1.074000e+05,53.573006,38151.847656,1.560000,2.493978,0.000000
75%,2007.000000,40195.003906,418.514847,40175.359375,1.373498,0.070833,97.799118,0.368750,815.064575,404.810089,29315.390625,5388.259033,53.125000,4.262826e+07,967.343674,23.120900,1131.237549,149.543597,169.332794,2.851986,0.002000,6.958872,0.005899,4.727100e+05,241.780823,47172.609375,9.962750,6.371659,0.006000
max,2021.000000,308704.218750,53368.628906,45161.207031,762.782654,1861.939819,100.000015,762.782654,46550.605469,40368.828125,40374.605469,56781.601560,100.000000,3.670250e+10,136131.468750,4345.990234,52181.949219,21388.600000,45161.207031,153.883406,1032.501221,1553.104980,1032.501221,1.638995e+07,40374.605469,53368.628906,4345.990234,1553.104980,1861.939819


In [61]:
df = df_original.copy()

## Scoring

#### Validation

In [62]:
df[
    df["Income group"].isnull()].query(
    "Year == 2020 and Entity_Category == 'Country'"
    )['Entity'].value_counts()

Montserrat                         1
Reunion                            1
Guadeloupe                         1
Saint Pierre And Miquelon          1
Anguilla                           1
Ussr                               1
Falkland Islands                   1
Western Sahara                     1
Bonaire Sint Eustatius And Saba    1
Mayotte                            1
Niue                               1
Cook Islands                       1
Venezuela                          1
Wallis And Futuna                  1
Kosovo                             1
Martinique                         1
Saint Helena                       1
French Guiana                      1
Name: Entity, dtype: int64

In [63]:
df.query("Year == 2020 and Entity_Category == 'Country'")["Income group"].value_counts(dropna=False)

High income            75
Lower middle income    54
Upper middle income    53
Low income             28
NaN                    18
Name: Income group, dtype: int64

In [64]:
df["Income group"].unique()

# Land area (sq. km)## 
# Annual CO2 emissions
## Income group

# nan,    -----------------> x0
# "High income" -----------> x1
# "Upper middle income", --> x1.5
# "Lower middle income", --> x1.75
# "Low income",  ----------> x2



# Score = (Annual CO2 emissions * Land Area) * Income Factor

array(['High income', 'Low income', 'Lower middle income', nan,
       'Upper middle income'], dtype=object)

### Map Income Group to Income Factor number

In [72]:
def income_group_mapping(text):
    if text == "High income":
        return 1
    if text == "Upper middle income":
        return 1.25
    if text == "Lower middle income":
        return 1.75
    if text == "Low income":
        return 2
    return 1

In [73]:
df["income_factor"] = df["Income group"].apply(lambda x: income_group_mapping(x))

In [74]:
df["income_factor"].describe()

count    13231.000000
mean         1.360630
std          0.377602
min          1.000000
25%          1.000000
50%          1.250000
75%          1.750000
max          2.000000
Name: income_factor, dtype: float64

In [75]:
df.head(2)

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels per capita (kWh),Oil Consumption - TWh,Economy,"Coal (TWh, direct energy)",Geo Biomass Other - TWh,Wind Generation - TWh,Fossil fuels (% equivalent primary energy),prod of Other renewables including bioenergy (TWh),Coal Production - TWh,Gas Production - TWh,"Gas (TWh, direct energy)",Per capita electricity (kWh),Renewables (% electricity),Annual CO2 emissions,Fossil fuels (TWh),Hydro Generation - TWh,Oil Production - TWh,Population density (people per sq. km of land area),Coal Consumption - TWh,Renewables per capita (kWh - equivalent),prod of Electricity from solar (TWh),Annual change in primary energy consumption (%),Solar Generation - TWh,Land area (sq. km),Gas Consumption - TWh,"Oil (TWh, direct energy)",prod of Electricity from hydro (TWh),Fossil fuels (% growth),prod of Electricity from wind (TWh),Entity_Category,income_factor,prioritization_score
34,Aruba,1961,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,645553.0,NaN,NaN,NaN,307.966667,NaN,NaN,NaN,NaN,NaN,180.0,NaN,NaN,NaN,NaN,NaN,Country,1.0,145249425.0
35,Aruba,1962,ABW,Latin America & Caribbean,High income,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,708942.0,NaN,NaN,NaN,312.411111,NaN,NaN,NaN,NaN,NaN,180.0,NaN,NaN,NaN,NaN,NaN,Country,1.0,159511950.0


### Calculate Prioritization Score

#### Score Calculation Formula
*Per Entity:*  
`prioritization_score` = `Annual CO2 emissions` * `Land area (sq. km)` * `income_factor`

In [95]:
entities_wo_area = df.drop_duplicates(['Code']).loc[df['Land area (sq. km)'].isna()]['Entity']

In [96]:
df.loc[df['Entity'].isin(
    entities_wo_area)][
    ['Entity', 'Year', 'Land area (sq. km)']].sort_values(
    'Land area (sq. km)')

,Entity,Year,Land area (sq. km)
11895,Luxembourg,2000,2574.46
11915,Luxembourg,2020,2574.46
11914,Luxembourg,2019,2574.46
11913,Luxembourg,2018,2574.46
11912,Luxembourg,2017,2574.46
...,...,...,...
21403,Wallis And Futuna,2016,NaN
21404,Wallis And Futuna,2017,NaN
21405,Wallis And Futuna,2018,NaN
21406,Wallis And Futuna,2019,NaN


In [101]:
df["prioritization_score"] = (
    df["Annual CO2 emissions"] * df['Land area (sq. km)']
)

In [102]:
df.sort_values(by='prioritization_score', ascending=False)

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels per capita (kWh),Oil Consumption - TWh,Economy,"Coal (TWh, direct energy)",Geo Biomass Other - TWh,Wind Generation - TWh,Fossil fuels (% equivalent primary energy),prod of Other renewables including bioenergy (TWh),Coal Production - TWh,Gas Production - TWh,"Gas (TWh, direct energy)",Per capita electricity (kWh),Renewables (% electricity),Annual CO2 emissions,Fossil fuels (TWh),Hydro Generation - TWh,Oil Production - TWh,Population density (people per sq. km of land area),Coal Consumption - TWh,Renewables per capita (kWh - equivalent),prod of Electricity from solar (TWh),Annual change in primary energy consumption (%),Solar Generation - TWh,Land area (sq. km),Gas Consumption - TWh,"Oil (TWh, direct energy)",prod of Electricity from hydro (TWh),Fossil fuels (% growth),prod of Electricity from wind (TWh),Entity_Category,income_factor,prioritization_score
3969,China,2020,CHN,East Asia & Pacific,Upper middle income,IBRD,23784.003906,7984.452637,China,NaN,135.625000,466.500000,83.505882,135.625000,22364.935547,1940.092163,NaN,5459.258789,28.087379,1.066789e+10,34232.882812,1321.708984,2265.163574,149.723553,22882.255859,4.056631,261.100006,2.541566,261.100006,9424702.9,3366.172607,NaN,1321.708984,1.602399,466.500000,Country,1.25,1.005417e+17
3968,China,2019,CHN,East Asia & Pacific,Upper middle income,IBRD,23499.349609,7915.177734,China,NaN,112.725227,405.299988,84.277794,112.725227,22154.439453,1767.414429,NaN,5277.177246,26.848570,1.048999e+10,33692.988281,1272.537964,2228.633545,149.367573,22694.042969,3.768683,224.000000,3.628755,224.000000,9424702.9,3083.768555,NaN,1272.537964,2.445459,405.299988,Country,1.25,9.886503e+16
3967,China,2018,CHN,East Asia & Pacific,Upper middle income,IBRD,23036.990234,7534.662109,China,NaN,93.725227,365.799988,85.251244,93.725227,21352.949219,1614.188599,NaN,5057.009277,25.610914,1.028999e+10,32888.707031,1198.886963,2201.838135,148.838644,22514.785156,3.460874,176.899994,3.951335,176.899994,9424702.9,2839.260010,NaN,1198.886963,2.862144,365.799988,Country,1.25,9.698009e+16
3963,China,2014,CHN,East Asia & Pacific,Upper middle income,IBRD,22106.707031,6142.516602,China,NaN,46.268219,159.762695,88.808533,46.268219,21680.763672,1311.808472,NaN,4183.150879,22.249449,9.985583e+09,30937.318359,1059.691650,2458.919189,145.560051,22911.171875,2.538375,23.512159,2.919328,23.512159,9424701.3,1883.631348,NaN,1059.691650,1.355195,159.762695,Country,1.25,9.411114e+16
3962,China,2013,CHN,East Asia & Pacific,Upper middle income,IBRD,21929.755859,5907.469727,China,NaN,37.132999,138.264130,90.179047,37.132999,22034.134766,1218.107178,NaN,3947.121094,20.129911,9.952744e+09,30523.664062,909.612244,2441.826416,144.645433,22897.412109,2.178754,8.373886,3.762960,8.373886,9424701.3,1718.785400,NaN,909.612244,3.184760,138.264130,Country,1.25,9.380164e+16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21406,Wallis And Futuna,2019,WLF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.643700e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country,1.00,NaN
21407,Wallis And Futuna,2020,WLF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.647400e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country,1.00,NaN
21550,Yemen,2021,YEM,Middle East & North Africa,Low income,IDA,NaN,NaN,"Yemen, Rep.",NaN,NaN,NaN,NaN,NaN,NaN,3.977500,NaN,NaN,NaN,NaN,NaN,NaN,32.096210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country,2.00,NaN
21688,South Africa,2021,ZAF,Sub-Saharan Africa,Upper middle income,IBRD,21795.685547,289.166656,South Africa,NaN,0.431334,8.188991,94.631485,0.431334,1541.734863,NaN,NaN,4113.701660,7.334341,NaN,1308.656494,1.401010,NaN,NaN,980.955627,0.785076,7.898076,0.518334,7.898076,NaN,38.534122,NaN,1.401010,0.948727,8.188991,Country,1.25,NaN


In [45]:
df[["Entity", "Code"]].drop_duplicates()

,Entity,Code
59263,Aruba,ABW
59529,Afghanistan,AFG
59795,Angola,AGO
60061,Albania,ALB
60327,United Arab Emirates,ARE
...,...,...
1910623,Eritrea,ERI
2278501,Gibraltar,GIB
2300047,Kosovo,OWID_KOS
2379049,Wallis And Futuna,WLF


In [46]:
df["income_factor"].value_counts(dropna=False)

1.25    1134540
1.75     826487
1.50     811063
2.00     438108
1.00     279303
Name: income_factor, dtype: int64

In [85]:
df_original.query("Year == 2020")

## Land area (sq. km)
## Annual CO2 emissions
## Income group

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels (% equivalent primary energy),Fossil fuels (% growth),Gas Production - TWh,Land area (sq. km),Unnamed: 0.1.1.1,Coal Consumption - TWh,Renewables per capita (kWh - equivalent),Wind Generation - TWh,prod of Other renewables including bioenergy (TWh),Fossil fuels per capita (kWh),Fossil fuels (TWh),Oil Production - TWh,Coal Production - TWh,Hydro Generation - TWh,Unnamed: 0.1.1,prod of Electricity from solar (TWh),prod of Electricity from hydro (TWh),Unnamed: 0.1,"Gas (TWh, direct energy)",Annual change in primary energy consumption (%),Geo Biomass Other - TWh,"Coal (TWh, direct energy)",Per capita electricity (kWh),Oil Consumption - TWh,Renewables (% electricity),Population density (people per sq. km of land area),Solar Generation - TWh,Gas Consumption - TWh,"Oil (TWh, direct energy)",Annual CO2 emissions,prod of Electricity from wind (TWh),Entity_Category
71,Afghanistan,2020,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,6.522300e+05,12892.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,12892.0,0.040000,0.620000,12892.0,NaN,NaN,NaN,NaN,20.014248,NaN,84.615395,59.684990,NaN,NaN,NaN,1.216029e+07,0.000000,Country
208,Africa,2020,NaN,NaN,NaN,NaN,89.685829,-6.705356,2312.026855,NaN,54.0,1157.901123,0.376615,21.302896,8.066416,3509.572021,4704.925781,3846.310547,1723.903687,146.373077,54.0,16.058805,146.373077,54.0,NaN,-5.539828,8.066416,NaN,628.423706,2011.052612,22.430740,NaN,16.058805,1535.972412,NaN,1.326044e+09,21.302896,Continent
264,Africa (Bp),2020,NaN,NaN,NaN,NaN,89.685829,-6.705356,2312.026855,NaN,110.0,1157.901123,NaN,21.302898,8.066416,NaN,4704.925781,3846.310547,1723.903687,146.373077,110.0,16.058805,146.373077,110.0,NaN,-5.539828,8.066416,NaN,NaN,2011.052612,22.430740,NaN,16.058805,1535.972412,NaN,NaN,21.302898,Bp?
354,Albania,2020,ALB,Europe & Central Asia,Upper middle income,IBRD,NaN,NaN,NaN,2.740000e+04,12893.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,12893.0,0.030000,5.280000,12893.0,NaN,NaN,NaN,NaN,1852.207153,NaN,100.000000,103.571131,NaN,NaN,NaN,4.534673e+06,0.000000,Country
459,Algeria,2020,DZA,Middle East & North Africa,Lower middle income,IBRD,99.702103,-5.827147,814.563721,2.381741e+06,166.0,4.816350,0.044457,0.007700,0.000000,14879.098633,652.463989,668.911072,NaN,0.049600,166.0,0.685000,0.049600,166.0,NaN,-5.824166,0.000000,NaN,1821.686035,211.635162,0.937777,18.411340,0.685000,436.012512,NaN,1.549955e+08,0.007700,Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27414,Western Sahara,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13179.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,13179.0,0.000000,0.000000,13179.0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Country
27685,World,2020,WLD,NaN,NaN,NaN,82.211304,-5.385285,38615.152344,1.299798e+08,6032.0,41963.964844,2.704402,1596.428223,703.916443,16523.912109,128800.398438,48508.015625,44068.472656,4345.990234,6032.0,846.229370,4345.990234,6032.0,38455.726562,-3.985941,703.916443,41963.964844,3429.322021,48380.710938,27.864630,59.731828,846.229370,38455.726562,48380.710938,3.480726e+10,1596.428223,World
27757,Yemen,2020,NaN,NaN,NaN,NaN,NaN,NaN,3.064353,NaN,12870.0,NaN,NaN,NaN,0.000000,NaN,NaN,43.814594,NaN,NaN,12870.0,0.500000,0.000000,12870.0,NaN,NaN,NaN,NaN,100.668922,NaN,15.384615,NaN,NaN,NaN,NaN,9.768313e+06,0.000000,Country
27840,Zambia,2020,ZMB,Sub-Saharan Africa,Low income,IDA,NaN,NaN,NaN,7.433900e+05,13180.0,NaN,NaN,NaN,0.160000,NaN,NaN,NaN,NaN,NaN,13180.0,0.150000,13.670000,13180.0,NaN,NaN,NaN,NaN,844.264526,NaN,87.484352,24.729894,NaN,NaN,NaN,6.572938e+06,0.000000,Country
